## Homework Assignment

A 3800 MWth rated BWR wishes to operate on an equilibrium 18-month cycle with an expected capacity factor (including refueling outage) of 95%. The total core loading consists of 800 fuel assemblies containing 180 MTU. The fuel region average discharge burnup limit is 44,000 MWD/MTU, and it is determined that a four-region core should be employed.

Determine the enrichment of feed fuel assemblies for each of the following approximations to the radial power distribution.

a) Flat power distribution.

b) Uneven power distribution, where the power distribution is given in the following table.

| Number of time batch burned at BOC | Avg. relative power of batch (batch power / avg. power over core) |
|------------------------------------|-------------------------------------------------------------------|
| 0                                  | 0.80                                                              |
| 1                                  | 1.30                                                              |
| 2                                  | 1.20                                                              |
| 3                                  | 0.70                                                              |


In [ ]:
from IPython.display import display, Math
import math

# Defining the given data
thermal_power = 3800  # in MWth
refueling_cycle = 18  # in months
capacity_factor = 95  # in percent
total_core_loading = 800  # number of fuel assemblies
fuel_loading_per_assembly = 180  # in MTU
burnup_limit = 44000  # in MWD/MTU

# Radial power distribution for uneven power distribution
power_distribution = {
    0: 0.80,
    1: 1.30,
    2: 1.20,
    3: 0.70
}

# Placeholder function to calculate enrichment for flat power distribution
def calculate_enrichment_flat(core_loading, burnup_limit):
    # Placeholder for enrichment calculation for flat power distribution
    # Implement based on specific requirements
    
    return enrichment_flat

# Placeholder function to calculate enrichment for uneven power distribution
def calculate_enrichment_uneven(core_loading, burnup_limit, power_distribution):
    # Placeholder for enrichment calculation for uneven power distribution
    # Implement based on specific requirements
    
    return enrichment_uneven

# Calculate enrichment for both cases
enrichment_flat = calculate_enrichment_flat(total_core_loading, burnup_limit)
enrichment_uneven = calculate_enrichment_uneven(total_core_loading, burnup_limit, power_distribution)

print(f"Enrichment for Flat Power Distribution: {enrichment_flat}")
print(f"Enrichment for Uneven Power Distribution: {enrichment_uneven}")